# Day 29: Model Governance Policy

In this lab, we will build a **Governance Policy** and automatically check if our `SafetyCase` (from Day 24) complies with it.
This simulates a "Gatekeeper" system where models cannot be deployed unless they pass specific checks.

In [ ]:
import sys
import os

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.governance.policy import PolicyRule, GovernancePolicy, ComplianceChecker
from src.assurance.safety_case import SafetyCase, Claim, Evidence, Argument

## 1. Define Policy Rules

Let's define a "Standard Deployment Policy" with two rules:
1.  **Must have Claims**: The safety case cannot be empty.
2.  **All Claims Verified**: Every claim must be in the "Verified" state.

In [ ]:
def check_has_claims(case: SafetyCase) -> bool:
    return len(case.claims) > 0

def check_all_verified(case: SafetyCase) -> bool:
    case.evaluate() # Ensure status is up to date
    for claim in case.claims.values():
        if claim.status != "Verified":
            return False
    return True

rules = [
    PolicyRule(id="R1", description="Safety Case must have at least one claim", check_func=check_has_claims),
    PolicyRule(id="R2", description="All claims must be Verified", check_func=check_all_verified)
]

policy = GovernancePolicy(name="Standard Deployment Policy", rules=rules)

## 2. Create Compliant Safety Case

We create a case that meets all criteria.

In [ ]:
compliant_case = SafetyCase(title="Compliant Model", version="1.0")

# Claim + Evidence + Argument -> Verified
c1 = Claim("C1", "Model is safe")
e1 = Evidence("E1", "Test Report", {"score": 1.0})
a1 = Argument("A1", "Scores are perfect", ["E1"])
c1.arguments.append(a1)

compliant_case.add_claim(c1)
compliant_case.add_evidence(e1)

## 3. Create Non-Compliant Safety Case

We create a case with a claim but missing evidence (so it won't be Verified).

In [ ]:
non_compliant_case = SafetyCase(title="Risky Model", version="0.1")

c2 = Claim("C2", "Model is robust")
# No evidence added!
non_compliant_case.add_claim(c2)

## 4. Check Compliance

Run the checker against both cases.

In [ ]:
checker = ComplianceChecker()

report_1 = checker.check(compliant_case, policy)
print(report_1)

print("\n---\n")

report_2 = checker.check(non_compliant_case, policy)
print(report_2)